### **ETL process (Extract, Transform & Load) - Plataforma de videojuegos Stream**
#### **DATOS: User_Reviews**

Empezaremos por entender, transfomar y disponibilizar, encontrando problemas en los datos, aplicando tecnicas de limpieza y preprocesamiente. Finalmente, almacenaremos los datos transformatos para futuras exploraciones.  

Entenderemos el contexto y la informacion necesaria para nuestro analisis, tratando consistencia a esos datos.

**Nota: es necesario instalar las siguientes librerias dentro del entorno:**

*`pip install pandas numpy gdwon langdetect nltk` *




#### **Importamos las librerias necesarias**

In [6]:
import pandas as pd #para el analisis de tablas tabulares

import numpy as np # Para trabajar con arrays y matrices

import os # OS module ayuda a interactuar directamente con el sistema operativo

import gdown # facilita la descarga de archivos desde Google Drive utilizando su ID

import json # Python admite formato JSON para intercambiar datos de manera efectiva.

from langdetect import detect # Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto

import nltk # NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano

from nltk.sentiment import SentimentIntensityAnalyzer # SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos

'''
Al descargar 'vader_lexicon' puede devolver un error, verificación del certificado SSL al descargar el recurso desde el 
servidor NLTK. Porque el entorno de Python no puede verificar la validez del certificado SSL utilizado por el servidor NLTK. 
'''
import ssl # Por esto importamos SSL
ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('vader_lexicon') # Nota: Este comando descarga el modelo de análisis de sentimiento de NLTK y solo debe ejecutarse la primera vez que se utiliza.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/cristian/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### **1. Función de Carga y Transformación del Archivo JSON Original**
Esta función realiza varios pasos para reformatear el archivo original, que tiene una estructura de diccionario de Python, en un formato JSON más adecuado. Aquí está lo que hace:

1. Verifica si el archivo original (en formato .json) ya está descargado. Si no lo está, lo descargará desde Google Drive, ya que el archivo no está en el formato correcto.

2. Aplica un formato adecuado al archivo para que quede un diccionario de Python dentro de un solo objeto JSON.

3. Guarda esos datos en un archivo llamado 'user_reviews_Estructurado.json'. y lo vamos a guardar como un data frame 'df_UserReviews'

In [7]:
def descargar_leer_json(url):
    """
    Descarga un archivo JSON desde la URL proporcionada, lo guarda localmente
    con el nombre 'datos_originales.json', lo convierte en registros y lo lee en un DataFrame de pandas.

    Parameters:
    - url (str): URL del archivo en Google Drive.

    Returns:
    - df (pd.DataFrame): DataFrame de pandas con los datos del archivo JSON.
    """

    # Descargar el archivo JSON si no existe localmente
    if not os.path.exists('user_reviews.json'):
        gdown.download(url, 'user_reviews.json', quiet=False)

    # Leer el archivo JSON descargado y convertirlo en texto
    with open('user_reviews.json', 'r', encoding='utf-8') as f:
        data = f.readlines()

    # Inicializar una lista vacía para almacenar los datos transformados
    transformed_data = []

    # Iterar sobre las líneas del archivo
    for line in data:
        # Eliminar los caracteres de nueva línea
        line = line.strip()
        # Convertir la línea en un diccionario de Python
        entry = eval(line)
        # Agregar la entrada transformada a la lista
        transformed_data.append(entry)

    # Escribir los datos transformados en un nuevo archivo JSON limpio
    with open('user_reviews_Estructurado.json', 'w') as json_file:
        json.dump(transformed_data, json_file, indent=4)

    # Leer el JSON transformado en un DataFrame de pandas
    df = pd.read_json('user_reviews_Estructurado.json')

    return df

### Llamamos la funcion para abrir el archivo

In [8]:
#URL del archivo en Google Drive y nombre del archivo local
url = 'https://drive.google.com/uc?export=download&id=1xQRf70-WS18UgjzAGKyJeTI30_f8wYyk'

#Utilizar la función (descargar_leer_json) para descargar y leer el JSON
df_UserReviews = descargar_leer_json(url)

In [9]:
# traemos una vista rapida de la tabla
df_UserReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### **# Analisis Primario**
1. el Dataframe User_reviews esta compuesto de 3 columnas 'user_id', 'user_url', y 'reviews'

2. la columna **'reviews'** contiene informacion valiosa para nuestro estudio. Contiene datos en un diccionario estructurado en formato json e incluye informacion como reseñas y fecha de publicacion, entre otras.

3. vamos a dividir la informacion de La columna **'reviews'** en diferentes columnas para separarla para facilitar el analisis. 

### **2. Desanidamos los datos de la columna 'reviews' y los dividimos en diferentes columnas**

In [10]:
# Cargar el archivo user_reviews_Estructurado.json en un DataFrame
df_from_json = pd.read_json('user_reviews_Estructurado.json')

# Aplana la estructura JSON y crea un nuevo DataFrame
user_reviews_TF = pd.json_normalize(df_from_json.to_dict('records'), record_path=['reviews'], meta=['user_id', 'user_url'])

user_reviews_TF.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [11]:
# Lista con el orden deseado de las columnas
column_order = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']

# Reordenar las columnas del DataFrame
user_reviews_TF = user_reviews_TF[column_order]

# Ver los primeros registros del DataFrame resultante
user_reviews_TF.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [12]:
# Obtener información general del DataFrame
print("\nInformación general del DataFrame:")
user_reviews_TF.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


### **# Insights**
1. Las columnas tienen tipos de datos **'object'** y **'boolean'**.
   
2. la columna **'recommended'** es util para observar la distribucion de recomendaciones en este conjunto de datos

In [13]:
# Calcula valores en la columna 'recommend' 
conteo_por_recomendaciones = user_reviews_TF['recommend'].value_counts()
porcentaje_recomendaciones = user_reviews_TF['recommend'].value_counts(normalize=True) * 100
#porcentaje_recomendaciones = porcentaje_recomendaciones.round(2).astype(str) + '%'

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_recomendaciones = pd.DataFrame({
    'Conteo': conteo_por_recomendaciones,
    'Porcentaje': porcentaje_recomendaciones.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_recomendaciones = resumen_recomendaciones.sort_values(by='Conteo', ascending=False)
resumen_recomendaciones

,Conteo,Porcentaje
recommend,,
True,52473,88.48%
False,6832,11.52%


### **3. Limpieza Y preprocesamiento de los datos**

1. No hay necesidad de hacer procesos para eliminar Nulos, ya que el dataframe no cuenta con valores Nulos

2. Eliminamos columnas innecesarias

3. Realizaremos analisis de texto para determinar el idioma de las reseñas 

4. Categorizaremos las reseñas por medio de analisis de sentimiento 

5. Crearemos columnas derivadas de los datos

In [14]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame (No es necesario)
user_reviews_TF.replace(['', 'null', 'None'], np.nan, inplace=True)

In [15]:
# Filas donde todas las columnas especificadas tienen valores nulos.

columnas_con_nulos = user_reviews_TF.columns[user_reviews_TF.isnull().any()]
nulos_por_columna = user_reviews_TF[columnas_con_nulos].isnull().sum()

# Mostrar columnas con nulos y cantidad de nulos debajo de cada una
for columna in columnas_con_nulos:
    print(f"Columna: {columna}")
    print(f"Nulos: {nulos_por_columna[columna]}")

Columna: funny
Nulos: 51154
Columna: last_edited
Nulos: 53165
Columna: review
Nulos: 30


In [16]:
# Se establece umbral del 80% para decidir que columnas eliminar por valores nulos
Treshold_nulos = 0.8

# Calcula el porcentaje de valores nulos por columna
porcentaje_nulos = user_reviews_TF.isnull().mean()
  
# Filtra las columnas que superan el umbral
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > Treshold_nulos]

# Muestra las columnas y su respectivo porcentaje de valores nulos
print("Columnas con más del {}% de valores nulos (candidatas a eliminar):".format(Treshold_nulos * 100))
for columna, porcentaje in columnas_a_eliminar.items():
    print("{}: {:.2%}".format(columna, porcentaje))

Columnas con más del 80.0% de valores nulos (candidatas a eliminar):
funny: 86.26%
last_edited: 89.65%


In [17]:
# Eliminamos las columnas que pasan el umbral establecido para valores nulos 
user_reviews_TF.drop(columns=columnas_a_eliminar.index, inplace=True)

In [18]:
# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes
user_reviews_TF.drop(['helpful','user_url'], axis=1, inplace=True)

In [19]:
# Se buscan registros duplicados
user_reviews_TF.sort_values('user_id')
filas_duplicadas = user_reviews_TF[user_reviews_TF.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
filas_duplicadas.count()

user_id      1736
posted       1736
item_id      1736
recommend    1736
review       1736
dtype: int64

In [20]:
# Se eliminan los registros duplicados basándome en múltiples columnas 
duplicados_eliminados = user_reviews_TF.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = filas_duplicadas.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1736


In [21]:
user_reviews_TF.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


### **# Insights**
La columna **'posted'** contiene fechas de esas reviews, vamos a extraerlos 

1. vamosa  crear una nueva columna de tipo entero llamada **'year'**

2. vamos a extrar esa informacion de la columna **'posted'**

3. Se eliminara la columna **'posted'** por optimizacion
   
4. Tendremos un atributo en cada review con respecto al año, facilitando analisis de tendencias en el tiempo


In [22]:
# Visualizar los 5 primeros datos de la columna 'posted'
unique_posted_values = user_reviews_TF['posted'].head().unique()

for value in unique_posted_values:
    print(value)

Posted November 5, 2011.
Posted July 15, 2011.
Posted April 21, 2011.
Posted June 24, 2014.
Posted September 8, 2013.


### **# Insights**
La columna 'posted' contenia algunos valores faltantes en cuanto al año. con lo cual vamos a abordarlo de la siguiente manera

1. Utilizaremos la tecnica de interpolacion para llenar valores faltantes en la columna **'year'**, ya que este proceso es util cuando se tiene una relacion secuencial.

2.Elegimos utilizar **la interpolacion lineal por grupo** en cambio de aplicar **la interpolacion linear global**, ya que captura las posibles variaciones en la temporalidad entre diferentes juegos; tambien se alinea con la naturaleza de los datos, ya que la temporalidad varia entre juegos. 

3. Aplicaremos una agrupacion por juego, utilizando su identificador en la columna **'item_id'**  y asi tener una relacion de estas reseñas con relacion al tiempo con sus respectivos juegos

4. Aplicaremos **interpolacion lineal por grupo** en relacion a cada 'item_id'. Asi, estimaremos años faltantes usando la relacion con los años conocidos de el mismo juego.

5. Limpiaremos nuestro dataset de valores nulos despues de aplicar la interpolacion lineal por grupo, y llenaremos estos espacios con la mediana de la columna **'year'**

6. Con el paso anterior aseguramos que nuesto dataframe mantengauna coherencia de temporalidad necesaria para futuros analisis.

In [23]:
# Convertimos la columna 'posted' a tipo datetime
user_reviews_TF['posted'] = pd.to_datetime(user_reviews_TF['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Creamos la columna 'year' a partir de 'posted'
user_reviews_TF['year'] = user_reviews_TF['posted'].dt.year.astype('Int64')

# Ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
user_reviews_TF = user_reviews_TF.sort_values(['item_id', 'year'])

# Rellenar valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
user_reviews_TF['year'] = user_reviews_TF.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Si aún hay valores nulos después de la interpolación, se llenan con la mediana. 
user_reviews_TF['year'] = user_reviews_TF['year'].fillna(user_reviews_TF['year'].median())

# Se eliminan las columnas 'posted' y 'user_id' que ya no son de utilidad
user_reviews_TF.drop(['posted', 'user_id'], axis=1, inplace=True)
user_reviews_TF.head()

/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_2402/2550706360.py:11: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  user_reviews_TF['year'] = user_reviews_TF.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)


,item_id,recommend,review,year
5331,10,True,this game is the 1# online action game is awes...,2011
22702,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,10,True,:D,2012
43134,10,True,Good Game :D,2012
24137,10,True,jueguenlooooooo,2013


### **3.1 Analisis de lenguaje**

Evaluando la columna 'review' podemos calcular la cantidad de reseñas por lenguage determinando:

1. El proceso de analisis de sentimiento se realiza en todo el conjunto de datos?, o

2. Es necesario aplicar un filtro especifico por idioma? 



In [24]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None


# Aplicar la función para detectar idioma y crear una nueva columna 'language'
user_reviews_TF['language'] = user_reviews_TF['review'].apply(detectar_idioma)


# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = user_reviews_TF['language'].value_counts()
porcentaje_por_idioma = user_reviews_TF['language'].value_counts(normalize=True) * 100


# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,44444,77.95%
pt,2111,3.7%
es,1243,2.18%
de,1115,1.96%
so,994,1.74%


In [25]:
# EDA information <<<--- 

# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
user_reviews_TF['idioma_completo'] = user_reviews_TF['language'].map(mapeo_idiomas)

# Crear un DataFrame con el resumen de idiomas
resumen_idiomas = user_reviews_TF['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)

# Calcular el porcentaje
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(user_reviews_TF)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)

# Guardar en un archivo CSV
resumen_idiomas.to_csv('resumen_idiomas.csv', index=False)

In [26]:
resumen_idiomas

,Idioma,Conteo,Porcentaje
0,English,44444,77.20
1,Portugués,2111,3.67
2,Español,1243,2.16
3,German,1115,1.94
4,Somali,994,1.73


In [27]:
# Filtrar los registros donde language no es igual a 'en' 
user_reviews_TF = user_reviews_TF[user_reviews_TF['idioma_completo'] == 'English']

# Borrar la columna 'idioma_completo'
user_reviews_TF = user_reviews_TF.drop('idioma_completo', axis=1)

### **# Insights**
1. Utilizaremos los registros en ingles unicamente, ya que tenemos mas representacion, generando mas precision para nuestro modelo futuro y optimizacion de recursos computacionales. 

2. Al selececionar un metodo de analisis de sentimiento es necesario considerar factores como:
    - Precision del metodo
    - Facilidad de su implementacion 
    - eficiencia de recursos computacionales

3. Elegimos utilizar **la biblioteca NLTK (Natural Language Toolkit)** por los siguientes factores: 
   1. Facilidad en su implementacion 
   2. Eficiencia compuesta
   3. Polaridad compuesta
   4. Niveles aceptables de precision

### **Feature Engineering- NLP**

1. A partir de la columna **'review'** crearemos una columna sentiment_analysis usando **NLP** con los siguientes parametros:
   1. '0' = malo
   2. '1' = neutral
   3. '2' = positivo
   4. Al faltar una reseña asignaremos el valor de '1' por defecto

2. Eliminaremos las columnas de review despues de ejecutar el proceso de **NLP**, son innecesarias para el analisis. 

In [28]:
# Convertimos todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
user_reviews_TF.loc[:, 'review'] = user_reviews_TF['review'].str.lower()

#Eliminación de caracteres especiales
user_reviews_TF['review'] = user_reviews_TF['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
user_reviews_TF.loc[:, 'review'] = user_reviews_TF['review'].str.replace('[^\w\s]', '', regex=True)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\w'
/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_2402/792922066.py:5: SyntaxWarning: invalid escape sequence '\s'
  user_reviews_TF['review'] = user_reviews_TF['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)
/var/folders/jx/mgrpvh4n2f3b1l736sh02rpw0000gn/T/ipykernel_2402/792922066.py:8: SyntaxWarning: invalid escape sequence '\w'
  user_reviews_TF.loc[:, 'review'] = user_reviews_TF['review'].str.replace('[^\w\s]', '', regex=True)


In [29]:
# Análisis de sentimiento para categorizar las reseñas columna 'review' 

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['review','compound_score'], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [30]:
# Llamar a la función analyze_sentiments
user_reviews_TF, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(user_reviews_TF)

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31486     70.84%
0                     8285     18.64%
1                     4673     10.51%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [31]:
# Borrar la columna 'language'
user_reviews_TF = user_reviews_TF.drop('language', axis=1)
user_reviews_TF

,item_id,recommend,year,sentiment_analysis
5331,10,True,2011,2
45506,10,True,2013,1
7801,10,True,2014,2
7967,10,True,2014,2
8519,10,True,2014,2
...,...,...,...,...
51725,99900,True,2015,2
53065,99900,False,2015,2
12393,99910,True,2011,2
53052,99910,False,2014,0


### 4. Guardamos los datos limpios
Guardamos estos archivos en los siguientes formatos para disponibilizarlos. CSV

In [32]:
# Los archivos se almacenan en local 
#user_reviews_TF.to_csv('user_reviews_limpios.csv', index=False)
#user_reviews_TF.to_json('user_reviews_limpios.json', orient='records', lines=True)
user_reviews_TF.to_parquet('../Dataframes/Dataframes_limpios/user_reviews_limpios.parquet', index=False)